# This jupyter notebook is used to pre-processing training data development for capstone 1 

The following steps are just repeated from steps of data wrangling. If you have any questions/concerns, please review here: <br>
https://github.com/jiaqixu/Springboard/blob/master/Capstone/Capstone1/data_wrangling_capstone1.ipynb

In [1]:
import os 
import pandas as pd
from datetime import datetime,date
from calendar import monthrange
import numpy as np
import math
from matplotlib import style
import matplotlib.pyplot as plt
from random import sample
from scipy import stats
import warnings
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline


#os.getcwd()
#deprecate runtime warning
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
prefix_market = 'sp500'
#prefix_market = 'FTSE100'
base_os = 'G:\Data\Machine_Learning_Library\spring_board_data_science\Capstone\Capstone1\data'
#os1 = os.path.join(base_os, prefix_market,'cleaned_data')
#os2 = os.path.join(base_os, prefix_market,'download_data_revise')
start_date = '2005-01-01'
end_date = '2020-01-01'
##load all data
filename = prefix_market + "_matrix_utd.csv"
origdat_matrix = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='symbol')
origdat_matrix.columns = origdat_matrix.columns.astype('datetime64[ns]')
print("The size of \"{}\" is: {}".format(filename, origdat_matrix.shape))

## define function to remove extreme price, for price of stocks above 5000 (this is data download problem)
def rm_extreme_price(dat, threshold_price = 5000):
    ##clean origdat
    tmp = dat.apply(lambda x:sum(x>threshold_price),axis=0)
    rm_list = []
    for i in range(tmp.shape[0]):
        if tmp[i]>0:
            rm_list.append(tmp.index[i])
    print("remove stocks:",rm_list)
    keep_list = [item for item in dat.columns if item not in rm_list]
    return(dat[keep_list])
    

filename = prefix_market + "_all_download_data_01-01-2005_12-31-2019_Adj Close.csv"
origdat = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
origdat = rm_extreme_price(origdat)
print("The size of \"{}\" is: {}".format(filename, origdat.shape))

filename = prefix_market + "_all_download_data_01-01-2005_12-31-2019_Adj Open.csv"
origdat_o = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
origdat_o = rm_extreme_price(origdat_o)
print("The size of \"{}\" is: {}".format(filename, origdat_o.shape))

filename = prefix_market + "_all_download_data_01-01-2005_12-31-2019_Volume.csv"
origdat_v = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
print("The size of \"{}\" is: {}".format(filename, origdat_v.shape))

#prefix_market='sp500'
filename = prefix_market + "_index_data_01-01-2000_05-20-2020.csv"
sp500 =  pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
print("The size of \"{}\" is: {}".format(filename, sp500.shape))


sp500 = pd.DataFrame(sp500['Close'])
sp500.columns = ['SP500']
#sp500.head()
origdat = origdat.merge(sp500, how = 'left', left_index=True, right_index=True)
#origdat.head()

origdat_ret = origdat.pct_change()
origdat_ret = origdat_ret[1:]
#origdat_ret = origdat
print("The size of return is:{}".format(origdat_ret.shape))
#origdat_ret.head()


##define set periods for train and trade
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, monthrange(year,month)[1])
    return date(year, month, day)

def diff_month(d1, d2):
    return -((d1.year - d2.year) * 12 + d1.month - d2.month)

def add_years(sourcedate, years):
    month = sourcedate.month
    year = sourcedate.year+years
    day = sourcedate.day
    return date(year, month, day)

def diff_year(d1,d2):
    return -((d1.year-d2.year))


#set for training periods
first_period1 = [datetime.strptime(item,'%Y-%m-%d') for item in ['2005-01-01', '2008-01-01', '2009-01-01']]
nrows1 = diff_year(datetime.strptime('2009-01-01','%Y-%m-%d'),datetime.strptime('2020-01-01','%Y-%m-%d'))+1
periods1=[]
for i in range(nrows1):
    periods1.append(list(map(lambda period: add_years(period,i), first_period1)))
    
periods1 = pd.DataFrame(periods1)
periods1.columns = ['train_start','trad_start','trad_end']
periods1 = periods1.apply(pd.to_datetime)
#print("Head of periods are:\n",periods1.head(),"\n")
#print("Tail of periods are:\n",periods1.tail(),"\n")

print("There are total {} training and trading sets.".format(periods1.shape[0]))
# define data check 
def data_quality(nrows,periods):
    add_symbol = "SP500"
    set_dict1 = {}
    for i in range(nrows):
        #choose date in periods1[i][1]
        sel_date0 = periods.iloc[i][0]
        sel_date = periods.iloc[i][1]
        
        ###move 1 year head since sub training stop 
        #sel_date = sel_date.replace(year = sel_date.year - 1)
        #print(sel_date)
        sel_stocks = origdat_matrix.index[origdat_matrix[sel_date]==1]
        #stocks should be the intersections of columnames of origdat_ret and set_stocks
        sel_list = list(set(origdat_ret.columns).intersection(set(sel_stocks)))
        sel_list.append(add_symbol)
        set_dat = origdat_ret[sel_list]
        # set in the date
        set_dat = set_dat[periods.iloc[i][0]:periods.iloc[i][1]]
        ##check the missing values
        keep_columns = []
        for item in set_dat.columns:
            if set_dat[item].isna().sum()/set_dat.shape[0]<=0.1:
                keep_columns.append(item)
        ##assert add_symbol must in keep_columns
        if add_symbol not in keep_columns:
            keep_columns.append(add_symbol)
        set_dat = set_dat[list(set(set_dat.columns).intersection(set(keep_columns)))]
        set_dict1[sel_date0] = sorted(set_dat.columns)
    
        #print("For period {}, there are total {} be selected!".format(sel_date,set_dat.shape[1]))
    return set_dict1
#set_dict1
set_dict1 = data_quality(nrows1,periods1) 

def dict_summ(set_dict):
    period_st = []
    num_stock = []
    for keys, value in set_dict.items():
        period_st.append(keys)
        num_stock.append(len(value))

    dict_summ = pd.DataFrame([period_st,num_stock]).T
    dict_summ.columns = ['start_time','num_stocks']
    return dict_summ

The size of "sp500_matrix_utd.csv" is: (711, 145)
remove stocks: ['TIE', 'BMC', 'PCL', 'POM', 'GAS', 'EMC', 'HOT', 'HAR', 'CHK', 'SCG']
The size of "sp500_all_download_data_01-01-2005_12-31-2019_Adj Close.csv" is: (3774, 626)
remove stocks: ['TIE', 'BMC', 'PCL', 'POM', 'GAS', 'EMC', 'HOT', 'HAR', 'CHK', 'SCG']
The size of "sp500_all_download_data_01-01-2005_12-31-2019_Adj Open.csv" is: (3774, 626)
The size of "sp500_all_download_data_01-01-2005_12-31-2019_Volume.csv" is: (3774, 636)
The size of "sp500_index_data_01-01-2000_05-20-2020.csv" is: (5128, 5)
The size of return is:(3773, 627)
There are total 12 training and trading sets.


**Since there are 12 periods, so there will be 12 sample dataset as train and trade, we pick up one as an example for pre-processing and training data development** <br>
The following steps are similar to several steps of EDA. If you have any questions/concerns, please review here: <br>
https://github.com/jiaqixu/Springboard/blob/master/Capstone/Capstone1/exploratory_data_analysis_capstone1.ipynb

In [2]:
train_ind = 2
cur_key = set_dict1[periods1.iloc[train_ind,0]]
start_date = periods1.iloc[train_ind,0]
end_date = periods1.iloc[train_ind,1]

cur_dat = origdat.loc[start_date:end_date,cur_key]
#cur_dat.fillna(method = 'ffill',inplace=True)
#fill na by 'ffill' method
cur_dat.fillna(method = 'ffill',inplace=True)
print(cur_dat.iloc[:5,:5])


m = list(range(1,21))
for k in range(2,13):
    m.append(k*20)
print(m)

def lag_return(dat,start_row,lag):
    return dat[start_row]/dat[start_row-lag]-1

def future_return(dat,start_row,fut):
    return dat[start_row+fut]/dat[start_row]-1


def create_feature_once(dat, start_ind, sel_feat):
    df = pd.DataFrame()
    #for i in range(len(sel_feat)):
    for ind in sel_feat:
        #print(ind)
        feat_ind = dat.apply(lag_return,args=(start_ind,ind),axis=0)
        feat_ind = feat_ind.to_frame()
        feat_ind = feat_ind.rename(columns={0:'f'+str(ind)})
        df = pd.concat([df,feat_ind],axis=1)
    return df

def create_target_once(dat,start_ind,sel_fut):
    target = dat.apply(future_return,args=(start_ind,sel_fut),axis=0)
    set_median = target.median()
    target = target.apply(lambda x:1 if x>set_median else 0)
    #target = target.to_frame()
    #target = target.rename(columns={0:'tar'+str(start_ind)})
    return(target)


def create_target_once2(dat,start_ind,sel_fut):
    target = dat.apply(future_return,args=(start_ind,sel_fut),axis=0)
    set_median = target.median()
    target = target.apply(lambda x:1 if x>0 else 0)
    #target = target.to_frame()
    #target = target.rename(columns={0:'tar'+str(start_ind)})
    return(target)

def create_feature_all(dat, sel_feat):
    print("create features for training data.")
    df = pd.DataFrame()
    total_dat = dat.shape[0]
    end_ind = total_dat - 1
    ind = max(sel_feat)
    #n =0 
    while ind<=end_ind-1:
        if ind%40==0:
            print(ind, end = " ")
        df_feat = create_feature_once(dat,start_ind= ind, sel_feat=sel_feat)
        #if n==0:
            #df_feat = create_feature_once(cur_dat,start_ind= ind, sel_feat=m)
        #    df = df_feat
        #else:
        df = pd.concat([df,df_feat],axis=0)
        #    df = np.hstack([df,df_feat])
        ind = ind + 1
    print("\n")
    return(df)

def create_target_all(dat,sel_fut,sel_feat):
    print("create classification for target data.")
    df = pd.DataFrame()
    total_dat = dat.shape[0]
    end_ind = total_dat - 1
    ind = max(sel_feat)
    while ind<=end_ind-sel_fut:
        if ind%40==0:
            print(ind, end = " ")
        df_target = create_target_once(dat,start_ind= ind, sel_fut=sel_fut)
        #df_target = create_target_once2(cur_dat,start_ind= ind, sel_fut=sel_fut)
        df = pd.concat([df,df_target],axis=0)
        ind = ind + 1
    print("\n")
    return(df)


# create features
df_feat = create_feature_all(dat = cur_dat, sel_feat=m)
#print(df_feat.head(5))
print("The size of train data set is:", df_feat.shape)
print("The head of train data set is:\n")
print(df_feat.head(5))

df_target = create_target_all(cur_dat,sel_fut=1,sel_feat=m)
print("The size of target data set is:", df_target.shape)
print("The head of target data set is:\n")
print(df_target.head(5))

                A  AABA   AAPL    ABC    ABT
date                                        
2007-01-03  22.40  6.98  10.39  18.34  13.73
2007-01-04  22.47  7.32  10.62  18.41  13.99
2007-01-05  22.26  7.56  10.55  18.19  13.99
2007-01-08  22.18  7.61  10.60  18.48  14.04
2007-01-09  22.21  7.52  11.48  18.42  14.17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
create features for training data.
240 280 320 360 400 440 480 520 560 600 640 680 720 

The size of train data set is: (197760, 31)
The head of train data set is:

            f1        f2        f3        f4        f5        f6        f7  \
A    -0.017406 -0.021057 -0.004200 -0.041633 -0.044722 -0.043180 -0.040469   
AABA -0.013534 -0.019432 -0.016492 -0.045124 -0.061516 -0.073446 -0.073446   
AAPL -0.007566 -0.002535  0.009837 -0.019518 -0.019925  0.002548  0.026522   
ABC  -0.019912 -0.032223 -0.028509 -0.064908 -0.066877 -0.049356 -0.036957   
ABT  -

In [3]:
X = df_feat
#print(X.mean())
y = df_target
#scaler = preprocessing.StandardScaler().fit(X)
# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
#X=scaler.transform(X)
y = np.ravel(y)

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,shuffle=True, random_state=42)
print("The data size for X_train is:", X_train.shape)
print("The data size for y_train is:", y_train.shape)
print("The data size for X_test is:", X_test.shape)
print("The data size for y_test is:", y_test.shape)

The data size for X_train is: (158208, 31)
The data size for y_train is: (158208,)
The data size for X_test is: (39552, 31)
The data size for y_test is: (39552,)


### StandardScaler for X_train and X_test

In [4]:
X_train.fillna(X_train.mean(),inplace=True)
X_test.fillna(X_test.mean(),inplace=True)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X.columns, index =X.index[:round(X.shape[0]*0.8)]) 
X_test = pd.DataFrame(X_test, columns=X.columns, index =X.index[round(X.shape[0]*0.8):]) 
y_train = pd.DataFrame(y_train, columns=["Target"], index =X.index[:round(X.shape[0]*0.8)])
y_test = pd.DataFrame(y_test, columns=["Target"], index =X.index[round(X.shape[0]*0.8):])
X_train.to_csv("Train_set"+str(train_ind)+"_Feature.csv")
y_train.to_csv("Train_set"+str(train_ind)+"_Target.csv")
X_test.to_csv("Test_set"+str(train_ind)+"_Feature.csv")
y_test.to_csv("Test_set"+str(train_ind)+"_Target.csv")

G:\Data\Machine_Learning_Library\learn_others\learn_from_others\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=2000, max_depth=30, max_features = 'log2', max_samples = 700, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Training accuracy:", round(clf.score(X_train,y_train),4))
print("Prediction accuracy:", round(clf.score(X_test,y_test),4))
cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion matrix:\n", pd.DataFrame(cnf_matrix))